In [3]:
import pandas as pd
import numpy as np
import sys
import gc
from sklearn.ensemble import RandomForestClassifier

from keras.models import Sequential, model_from_json
from keras.layers.core import Dense, Activation
from keras.optimizers import Adagrad

import utils
from grid import RoadGrid

Using Theano backend.
Using gpu device 0: GeForce GTX 780 (CNMeM is disabled, cuDNN not available)


In [1]:
def encode_rncci(rncid, ci):
    return rncid*100+ci

def decode_rncci(rncci):
    rncid = rncci / 100
    ci = rncci % 100
    return rncid, ci

def get_rncci_dict(tr_data):
    # Get all RNC_CI
    rncci_set = set()
    for i in xrange(1,7):
        rncci_set |= set(encode_rncci(tr_data['RNCID_%d'%i].values.astype(int), tr_data['CellID_%d'%i].values.astype(int)))
    rncci_set.remove(encode_rncci(-999,-999))
    rncci_dict = dict(zip(list(rncci_set), range(len(rncci_set))))
    
    eng_para = utils.get_4g_engpara()
    
    eng_para = utils.get_4g_engpara()
    rncci = encode_rncci(eng_para.LAC.values, eng_para.CI.values)
    lnglat = eng_para[[u'经度', u'纬度']].values
    rncci_lnglat_dict = dict(zip(rncci, lnglat))
    rncci_lnglat = [tuple(rncci_lnglat_dict[x]) for x in list(rncci_set)]
    
    return rncci_dict, rncci_lnglat, list(rncci_set)

def get_train(tr_data, rncci_dict):
    tr_label = tr_data[['Longitude', 'Latitude']].values

    # Load Engineering Parameter
    eng_para = utils.get_4g_engpara()
    print '数据集大小: %d' % len(tr_data)

    # Grid Data
    tr_time, tr_feature, tr_label_ = utils.make_rf_dataset(tr_data, eng_para)
    rg = RoadGrid(tr_label_.values, 10)
    tr_label_ = np.array(rg.transform(tr_label_.values, False))
    print '格子数量: %d' % rg.n_grid

    # Grid Statistics
    dense_feature = [1] * len(tr_feature)
    sparse_feature = [1] * len(tr_feature)
    for i in xrange(rg.n_grid):
        sub_feature = tr_feature[tr_label_ == i]
        bs_count = np.zeros(len(rncci_dict))
        bs_rscp = np.zeros(len(rncci_dict))
        bs_ecno = np.zeros(len(rncci_dict))
        for idx, row in sub_feature.iterrows():
            sp_rscp = np.zeros(len(rncci_dict))
            sp_ecno = np.zeros(len(rncci_dict))
            for j in xrange(1, 7):
                rncid = int(row['RNCID_%d'%j])
                ci = int(row['CellID_%d'%j])
                rscp = float(row['RSCP_%d'%j])
                ecno = float(row['EcNo_%d'%j])
                if rncid != -999 and ci != -999 and rscp != -999 and ecno != -999 and rscp != 0 and ecno != 0:
                    dict_idx = rncci_dict[encode_rncci(rncid, ci)]
                    bs_count[dict_idx] += 1
                    bs_rscp[dict_idx] += (rscp+140) # add constant scalar to be a positive number
                    bs_ecno[dict_idx] += (ecno+30)
                    sp_rscp[dict_idx] = (rscp+140)
                    sp_ecno[dict_idx] = (ecno+30)
            sparse_feature[idx-1] = list(np.hstack((sp_rscp, sp_ecno)))
        radio_map = np.hstack((bs_rscp / bs_count, bs_ecno / bs_count))
        nan_mask = np.array([np.isnan(x) for x in radio_map])
        nan_idx = np.where(nan_mask == True)[0]
        radio_map[nan_idx] = 0
        for idx, row in sub_feature.iterrows():
            dense_feature[idx-1] = list(radio_map)

    dense_feature = np.asarray(dense_feature)
    sparse_feature = np.asarray(sparse_feature)
    return sparse_feature, dense_feature

def build_network(n_dim, n_hidden):
    model = Sequential()
    model.add(Dense(output_dim=n_hidden, input_dim=n_dim))
    model.add(Activation("relu"))
    model.add(Dense(output_dim=n_dim))

    return model

def to_dense_feature(tr_feature, rncci_lnglat, rncci_list):
    n_rncci = tr_feature.shape[1] / 2
    tr_rssi = tr_feature[:, :n_rncci] - tr_feature[:, n_rncci:]
    idx = np.argsort(tr_rssi,axis=1)[:,::-1]
    full_feature = []
    for i in xrange(len(tr_feature)):
        p_feature = []
        for j in xrange(7):
            rscp_ = tr_feature[i, idx[i,j]]
            ecno_ = tr_feature[i, idx[i,j]+n_rncci]
            bs_lon_, bs_lat_ = rncci_lnglat[idx[i,j]]
            rncid_, ci_ = decode_rncci(rncci_list[idx[i,j]])
            if j==6 or rscp_ == 0.:
                break
            p_feature += [rscp_, ecno_, bs_lon_, bs_lat_, rncid_, ci_]
        p_feature += [0]*(6-j)*6 + [j]
        full_feature.append(p_feature)
    full_feature = np.array(full_feature)
    return full_feature

In [4]:
tr_names = ['forward0', 'forward1', 'forward2', 'forward3','forwardbackward4', 'forwardbackward5', 'bu_1', 'bu_2']
te_names = ['backward2']

tr_data = utils.get_4g_data(tr_names)
te_data = utils.get_4g_data(te_names)

rncci_dict, rncci_lnglat, rncci_list = get_rncci_dict(pd.concat([tr_data, te_data]))

tr_sparse_feature, tr_radiomap = get_train(tr_data, rncci_dict)
te_sparse_feature, te_radiomap = get_train(te_data, rncci_dict)

数据集大小: 56949
格子数量: 7259
数据集大小: 4675
格子数量: 1549


In [ ]:
tr_dense_feature = to_dense_feature(tr_sparse_feature, rncci_lnglat, rncci_list)
te_dense_feature = to_dense_feature(te_sparse_feature, rncci_lnglat, rncci_list)

In [ ]:
tr_dense_feature1 = to_dense_feature(tr_sparse_feature, rncci_lnglat, rncci_list)
tr_dense_feature2 = to_dense_feature(tr_radiomap, rncci_lnglat, rncci_list)

In [ ]:
def normalize(data, is_train=True, mean_list = None, std_list = None):
    n, m = data.shape
    if is_train:
        mean_list = []
        std_list = []
        for i in range(m):
            mean_list.append(np.mean(data[:, i]))
            std_list.append(np.std(data[:, i]))
            data[:, i] = (data[:, i] - mean_list[i]) / std_list[i] if std_list[i] else 0
        return data, mean_list, std_list
    else:
        for i in range(m):
            data[:, i] = (data[:, i] - mean_list[i]) / std_list[i] if std_list[i] else 0
        return data
def denormalize(data, mean_list, std_list):
    n, m = data.shape
    for i in range(m):
        data[:, i] = (data[:, i] * std_list[i]) + mean_list[i]
    return data

In [ ]:
tr_sparse_feature_norm, tr_feature_mean, tr_feature_std = normalize(tr_sparse_feature, True)
tr_radiomap_norm, tr_radiomap_mean, tr_radiomap_std = normalize(tr_radiomap, True)

In [ ]:
n_hidden = 600
nb_epoch = 100
batch_size = 64

network = build_network(tr_sparse_feature.shape[1], n_hidden)
network.compile(loss='mean_squared_error', optimizer=Adagrad())
network.summary()
network.fit(tr_sparse_feature, tr_radiomap, nb_epoch=nb_epoch, batch_size=batch_size, validation_split=0.05, verbose=1)

In [ ]:
json_str = network.to_json()
f_json = open('S2D.json', 'w')
f_json.write(json_str)
f_json.close()
network.save_weights('S2D.weight', overwrite=True)

In [ ]:
f_json = open('S2D.json')
json_str = f_json.readline()
network = model_from_json(json_str)
network.load_weights('S2D.weight')

In [ ]:
tr_imputed =  to_dense_feature(network.predict(tr_sparse_feature), rncci_lnglat, rncci_list)
te_imputed =  to_dense_feature(network.predict(te_sparse_feature), rncci_lnglat, rncci_list)

In [ ]:
tr_imputed = network.predict(tr_sparse_feature).astype(int)
te_imputed = network.predict(te_sparse_feature).astype(int)

In [ ]:
tr_imputed = tr_imputed.astype(int)
te_imputed = te_imputed.astype(int)

In [ ]:
print np.mean(tr_imputed[:, -1])
print np.mean(te_imputed[:, -1])

In [9]:
tr_label = tr_data[['Longitude', 'Latitude']].values
rg = RoadGrid(tr_label, 30)
tr_label_ = np.array(rg.transform(tr_label, False))
te_label = te_data[['Longitude', 'Latitude']].values
print rg.n_grid

1841


In [8]:
tr_imputed = tr_sparse_feature.astype(int)
te_imputed = te_sparse_feature.astype(int)

In [10]:
est = RandomForestClassifier(
        n_jobs=-1,
        n_estimators = 50,
        max_features='sqrt',
        bootstrap=True,
        criterion='gini'
    ).fit(tr_imputed, tr_label_)

In [11]:
te_pred = est.predict(te_imputed)
te_pred = np.array([rg.grid_center[gid] for gid in te_pred])
error_imputed = [utils.distance(pt1, pt2) for pt1, pt2 in zip(te_pred, te_label)]
utils.report(error_imputed, '%d\t%d\t%d' % (len(tr_imputed), len(te_imputed), tr_imputed.shape[1]))

56949	4675	1224	51.08	28.10	43.40	62.30	108.70


In [ ]:
est.feature_importance_

In [ ]:
eng_para = utils.get_4g_engpara()
tr_time, tr_feature, tr_label_ = utils.make_rf_dataset(tr_data, eng_para)
te_time, te_feature, te_label_ = utils.make_rf_dataset(te_data, eng_para)
rg = RoadGrid(tr_label_.values, 30)

In [ ]:
tr_feature[['RSCP_%d'%i for i in xrange(1,7)]]

In [ ]:
tr_label = tr_data[['Longitude', 'Latitude']].values
rg = RoadGrid(tr_label, 30)
tr_label_ = np.array(rg.transform(tr_label, False))
te_label = te_data[['Longitude', 'Latitude']].values
print rg.n_grid

In [ ]:
est2 = RandomForestClassifier(
        n_jobs=-1,
        n_estimators = 50,
        max_features='sqrt',
        bootstrap=True,
        criterion='gini'
).fit(tr_feature, tr_label_)

In [ ]:
te_pred = est2.predict(te_feature)
te_pred = np.array([rg.grid_center[gid] for gid in te_pred])
error = [utils.distance(pt1, pt2) for pt1, pt2 in zip(te_pred, te_label)]
utils.report(error, '%d\t%d\t%d' % (len(tr_feature), len(te_feature), tr_feature.shape[1]))

In [ ]:
error

# 验证多基站精度

In [ ]:
# Load Training Data
tr_dnames = ['forward0', 'forward1', 'forward2', 'forward3']
tr_data = utils.get_4g_data(tr_dnames)
tr_label = tr_data[['Longitude', 'Latitude']].values

# Load Testing Data
te_dnames = ['backward2']
te_data = utils.get_4g_data(te_dnames)
te_label = te_data[['Longitude', 'Latitude']].values

# Load Engineering Parameter
eng_para = utils.get_4g_engpara()

print '训练集大小: %d' % len(tr_data)
print '测试集大小: %d' % len(te_data)

In [ ]:
tr_time, tr_feature, tr_label_ = utils.make_rf_dataset(tr_data, eng_para)
te_time, te_feature, te_label_ = utils.make_rf_dataset(te_data, eng_para)
rg = RoadGrid(tr_label_.values, 10)
#tr_label_ = np.array(rg.transform(tr_label_.values, False))

In [ ]:
def feature_names(n_bs):
    return np.array([['RNCID_%d'%i, 'CellID_%d'%i, 'EcNo_%d'%i, 'RSCP_%d'%i, u'经度%d'%i, u'纬度%d'%i] for i in xrange(1, n_bs+1)]).ravel()

In [ ]:
def part_train(n_bs, tr_feature, tr_label, te_feature, te_label):
    rg = RoadGrid(tr_label_.values, 30)
    tr_label = np.array(rg.transform(tr_label, False))
    
    # cut feature
    tr_feature = tr_feature[feature_names(n_bs)]
    te_feature = te_feature[feature_names(n_bs)]
    
    # remove train null
    for i in xrange(1, n_bs+1):
        tr_mask = (tr_feature['RNCID_%d'%i].values != -999) & (tr_feature['CellID_%d'%i].values != -999)
        tr_feature = tr_feature[tr_mask]
        tr_label = tr_label[tr_mask]
    
    
    #print '特征维度: %d' % tr_feature.shape[1]
    est = RandomForestClassifier(
        n_jobs=-1,
        n_estimators = 100,
        max_features='sqrt',
        bootstrap=True,
        criterion='gini'
    ).fit(tr_feature.values, tr_label)
    
    te_pred = est.predict(te_feature.values)
    te_pred = np.array([rg.grid_center[gid] for gid in te_pred])
    
    error = [utils.distance(pt1, pt2) for pt1, pt2 in zip(te_pred, te_label)]
    
    utils.report(error, '%d\t%d\t%d' % (len(tr_feature), len(te_feature), tr_feature.shape[1]))
    #return error

In [ ]:
for i in xrange(1, 7):
    part_train(i, tr_feature, tr_label, te_feature, te_label)

In [ ]:
part_train(2, tr_feature, tr_label, te_feature, te_label)

In [ ]:
tr_label

In [ ]:
te_feature.shape

In [ ]:
tr_feature['RNCID_1'].notnull().values

In [ ]:
t